In [1]:
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import shapely.speedups
import requests
import json
from census import Census
from us import states
import censusgeocode as cg
import pyproj
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from geopandas.tools import sjoin

<ipython-input-1-d7f2bc1e0341>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
# speeds up shapely calls
# shapely.speedups.enable()
transformer = pyproj.Transformer.from_crs("EPSG:2263",'epsg:4326')


In [3]:
# read geo data
zipfile1 = "zip://./data/cb_2018_34_tract_500k.zip"
zipfile2 = "zip://./data/cb_2018_36_tract_500k.zip"
tract_data1 = geopandas.read_file(zipfile1)
tract_data2 = geopandas.read_file(zipfile2)
tract_data = geopandas.GeoDataFrame(pd.concat([tract_data1,tract_data2]))
tract_data

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,34,023,004700,1400000US34023004700,34023004700,47,CT,648594,0,"POLYGON ((-74.29005 40.52265, -74.28748 40.521..."
1,34,023,006103,1400000US34023006103,34023006103,61.03,CT,2845215,9920,"POLYGON ((-74.49146 40.45320, -74.48557 40.457..."
2,34,023,006500,1400000US34023006500,34023006500,65,CT,5178980,530,"POLYGON ((-74.42388 40.44648, -74.42045 40.445..."
3,34,029,715902,1400000US34029715902,34029715902,7159.02,CT,3377179,201835,"POLYGON ((-74.19763 40.04472, -74.19664 40.050..."
4,34,029,721000,1400000US34029721000,34029721000,7210,CT,2260503,240040,"POLYGON ((-74.33147 40.02096, -74.32669 40.021..."
...,...,...,...,...,...,...,...,...,...,...
4901,36,047,059100,1400000US36047059100,36047059100,591,CT,192105,0,"POLYGON ((-73.94543 40.72552, -73.94357 40.725..."
4902,36,009,961300,1400000US36009961300,36009961300,9613,CT,413393835,2880671,"POLYGON ((-78.94409 42.02386, -78.93824 42.025..."
4903,36,013,035300,1400000US36013035300,36013035300,353,CT,96482169,10449194,"POLYGON ((-79.29492 42.45567, -79.29472 42.456..."
4904,36,093,020300,1400000US36093020300,36093020300,203,CT,490013,53188,"POLYGON ((-73.94188 42.82295, -73.94016 42.823..."


In [4]:
def clean_irrelevant(df):
    #Clean records with no coordinates and no address
    df.drop(df.loc[(df['Address Type']!='ADDRESS') & (df['X Coordinate (State Plane)'].isnull() | df['Latitude'].isnull())].index, inplace=True)
    return df

In [5]:
def get_coordinates(row):
    if (pd.notnull(row['X Coordinate (State Plane)']) and pd.isnull(row['Latitude'])):
        lat,long = transformer.transform(row['X Coordinate (State Plane)'], row['Y Coordinate (State Plane)'])
        row['Latitude'] = lat
        row['Longitude'] = long
        #What's the point of following statement?
        if pd.notnull(row['Latitude']) and pd.notnull(row["Longitude"]):
            lat = row['Latitude']
            long = row['Longitude']

I think the problem was that the Coordinate Reference System (CRS) was undefined for gdf. Added it and now it works.

In [6]:
for year in range(2010,2019):
    # read 311 data
    print(str(year))
    calls = pd.read_csv("./data/311_"+str(year)+".csv")
    tqdm_notebook.pandas(desc='Converting XY to long/lat...')
    calls = clean_irrelevant(calls)
    print('Shape after cleaning year '+str(year)+ str(calls.shape))
    calls['tract'] = calls.progress_apply(lambda row: get_coordinates(row), axis=1)
    # create geo df
    gdf = geopandas.GeoDataFrame(calls, geometry=geopandas.points_from_xy(calls.Longitude, calls.Latitude), crs="EPSG:4269")
    gdf = gdf[gdf['geometry'].is_valid].reset_index(drop=True)
    # print(tract_data.head())
    # print(gdf.shape, tract_data.shape)
    # print(gdf.head())
    # print(tract_data.head())
     
    # try spatial join
    # https://stackoverflow.com/questions/62506697/how-to-find-inside-which-multipolygon-of-a-geopandas-dataframe-a-point-lies
    pointInPolys = sjoin(gdf, tract_data, how='left', op='within')
    pointInPolys = pointInPolys.drop(columns=['X Coordinate (State Plane)','Y Coordinate (State Plane)','Location','tract', 'geometry', 'index_right', 'NAME', 'LSAD','ALAND', 'AWATER'])

    # Save file with tracts for each call
    pointInPolys.to_csv('./data/311Calls_tracts/calls_and_tracts_'+str(year)+'.csv', index=False)


2010


/Users/Yan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8,17,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/Yan/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Shape after cleaning year 2010(433797, 41)



2011


/Users/Yan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/Yan/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


...

/Users/Yan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8,17,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/Yan/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Shape after cleaning year 2017(585242, 41)



2018


/Users/Yan/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/Yan/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Shape after cleaning year 2018(650078, 41)
